# Projetando um PID

## Modelando o sistema

Este trabalho foi baseado no problema 3.17 do livro *Control Systems* de *Norman S. Nise*, Sétima edição.

![Modelando o vôo do míssil](img/missil.png)

O problema modela o vôo de um míssil, que está sujeito a quatro forças: empuxo (*thrust*), sustentação (*lift*),
arrasto (*drag*) e gravidade. O míssil voa com um ângulo de ataque, $\alpha$, do seu eixo longitudinal, criando sustentação. Para seguir um determinado rumo, o ângulo do corpo da vertical, $\phi$, é controlado rotacionando o motor na cauda. 

A função de transferência relaciona o ângulo do corpo, $\phi$, e sua posição angular, $\delta$, do motor na forma, como mostrado na equação abaixo:

\begin{align}
 \frac{\Phi(s)}{\delta(s)} = \frac{K_a s + K_b}{K_3 s^3 + K_2 s^2 + K_1 s + K_0} \label{eq:plant}
\end{align}

## Entendendo a planta

Desejamos projetar um controlador PID para controlar o míssil, conforme relacionado pelo diagrama de blocos abaixo:

![Diagrama de blocos](img/block-diagram.svg)

Para fins práticos, escolheremos os parâmetros da planta de forma arbitrária.

A resposta ao degrau e impulso da planta são mostrados abaixo:

In [97]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
from control import (TransferFunction, step_response, bode_plot,
                     impulse_response, series, feedback, rlocus)

ka, kb,  = [1, 5]
k3, k2, k1, k0 = [2, 50, 10, 5]

plant_tf = TransferFunction([ka, kb], [k3, k2, k1, k0])

In [98]:
def plot_step_response(tf, title='Step response', show_points=False):
    time, output = step_response(tf)
    fig = plt.figure()
    plt.plot(time, output)
    if show_points:
        plt.plot(time, output,'om')
    plt.title(title)
    plt.show()
    return time,output

time,output = plot_step_response(plant_tf, 
                                 title='Plant step response')

FigureCanvasNbAgg()

E também podemos visualizar a resposta em frequência da planta através do diagrama de *Bode*:

In [99]:
plt.figure()
plt.title("Bode plot")
mag, phase, omega = bode_plot(plant_tf)

FigureCanvasNbAgg()

## Ajustando controlador

### Método resposta a frequência de Ziegler-Nichols
A fim de aplicarmos este método, acharemos o ganho crítico do sistema. Para isso analisaremos o lugar das raízes:

In [21]:
loci = rlocus(plant_tf, Plot=True, PrintGain=True)

FigureCanvasNbAgg()

Porém, percebe-se que nosso sistema não tem um ganho crítico, podemos aumentá-lo indefinidamente. Logo, não podemos utilizar este método.

### Método resposta ao degrau de Ziegler-Nichols

Para este método, precisamos achar o ponto de inflexão da curva para podermos calcular dois parâmetros: $L$ e $\alpha$. Destes dois 
parâmetros, projetaremos um P, PI ou PID seguindo as regras da tabela:

\begin{array}{rr} \hline
\text{Controlador} & K &T_i&T_d\\ \hline
\text{P} &1/\alpha&&& \\ \hline
\text{PI} &0.9/\alpha&3L&& \\ \hline
\text{PID} &1.2/\alpha&2L&L/2& \\ \hline
\end{array}

E a seguinte função transferência:
\begin{align}
\frac{\Delta(s)}{E(s)} = K_c \left( 1 + \frac{1}{T_i s} + T_d s \right)
\end{align}

Para encontrar estes parâmetros, revisitaremos a resposta ao degrau 
e escolheremos o ponto de inflexão para traçar a tangente.

In [100]:
time, output = plot_step_response(plant_tf, 
                                  title='Plant step response',
                                  show_points=True)

FigureCanvasNbAgg()

Escolhemos o quinto ponto como ponto de flexão, e vamos tracejar a linha tangente a ele, através da sua derivada (discreta).

In [101]:
def derivate_around(x,y,index):
    return (y[index] - y[index - 1])/(x[index]- x[index - 1])

def tangent_line(x, y, index):
    return y[index]+derivate_around(x, y, index)*(x - x[index])

inflection_index = 5
plt.figure()
plt.title("Inflection point tangent line")
axes = plt.gca()
axes.set_xlim([0,25])
axes.set_ylim([-0.5,2])
plt.grid(True)
plt.plot(time,output,'b',time,tangent_line(time, output, inflection_index),'--r')
plt.show()

FigureCanvasNbAgg()

Da figura acima obtemos:

\begin{align*}
L=1.12\\
\alpha=0.22
\end{align*}

In [74]:
def ziegler_nichols_constants(l,alpha):
    return (1.2/alpha, 2*l, l/2)

l = 1.12
alpha = 0.22
k, ti, td = ziegler_nichols_constants(l, alpha)

(k, ti, td) 

(5.454545454545454, 2.24, 0.56)

Logo, temos:

\begin{align}
K_c &= 5.45 \\
T_i &= 2.24 \\
T_d &= 0.56 \\
\frac{\Delta(s)}{E(s)} &= 5.45 \left( 1 + \frac{1}{2.24 s} + 0.56 s \right) = \frac{6.83 s^2 + 12.2s +5.45}{2.24 s}
\end{align}

In [109]:
controller_tf = TransferFunction([k*ti*td, k*ti, k],[ti, 0])
system = feedback(series(controller_tf, plant_tf), 1)

plot_step_response(system)

FigureCanvasNbAgg()

(array([ 0.        ,  0.8304762 ,  1.66095239,  2.49142859,  3.32190479,
         4.15238099,  4.98285718,  5.81333338,  6.64380958,  7.47428577,
         8.30476197,  9.13523817,  9.96571436, 10.79619056, 11.62666676,
        12.45714296, 13.28761915, 14.11809535, 14.94857155, 15.77904774,
        16.60952394, 17.44000014, 18.27047634, 19.10095253, 19.93142873,
        20.76190493, 21.59238112, 22.42285732, 23.25333352, 24.08380972,
        24.91428591, 25.74476211, 26.57523831, 27.4057145 , 28.2361907 ,
        29.0666669 , 29.89714309, 30.72761929, 31.55809549, 32.38857169,
        33.21904788, 34.04952408, 34.88000028, 35.71047647, 36.54095267,
        37.37142887, 38.20190507, 39.03238126, 39.86285746, 40.69333366,
        41.52380985, 42.35428605, 43.18476225, 44.01523844, 44.84571464,
        45.67619084, 46.50666704, 47.33714323, 48.16761943, 48.99809563,
        49.82857182, 50.65904802, 51.48952422, 52.32000042, 53.15047661,
        53.98095281, 54.81142901, 55.6419052 , 56.4